In [1]:
!pip install torchaudio librosa transformers jiwer

import os
import torch
import torchaudio
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from jiwer import cer


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [2]:
# Use CPU as the device
device = torch.device("cpu")
print(f'Using device: {device}')


Using device: cpu


In [3]:
#Download and Prepare Data
data_path = './data'
if not os.path.exists(data_path):
    os.makedirs(data_path)

# Dictionary to hold datasets
datasets = {
    'train-clean-100': torchaudio.datasets.LIBRISPEECH(data_path, url="train-clean-100", download=True),
    'train-clean-360': torchaudio.datasets.LIBRISPEECH(data_path, url="train-clean-360", download=True),
    'test-clean': torchaudio.datasets.LIBRISPEECH(data_path, url="test-clean", download=True),
    'test-other': torchaudio.datasets.LIBRISPEECH(data_path, url="test-other", download=True)
}

100%|██████████| 5.95G/5.95G [03:03<00:00, 34.9MB/s]
100%|██████████| 21.5G/21.5G [12:01<00:00, 31.9MB/s]
100%|██████████| 331M/331M [00:11<00:00, 31.0MB/s]
100%|██████████| 314M/314M [00:09<00:00, 33.1MB/s]


In [4]:
# Load and Prepare Model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
model.to(device)  # Ensure model is set to CPU
model.eval()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/162 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-large-960h-lv60-self were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.maske

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (1-4): 4 x Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projec

In [5]:
#Define Speech Recognition Function
def speech_to_text(model, processor, waveform, sampling_rate):
    # Ensure waveform is mono and resample if necessary
    waveform = waveform.to(device)  # Move waveform to CPU
    if waveform.ndim > 1:
        waveform = waveform.mean(dim=0)
    if sampling_rate != 16000:
        resampler = torchaudio.transforms.Resample(orig_freq=sampling_rate, new_freq=16000)
        waveform = resampler(waveform)

    # Process waveform through the model
    inputs = processor(waveform, sampling_rate=16000, return_tensors="pt", padding=True)
    inputs = inputs.to(device)  # Ensure inputs are on CPU
    with torch.no_grad():
        logits = model(inputs.input_values, attention_mask=inputs.attention_mask).logits

    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.decode(predicted_ids[0])
    return transcription

In [6]:
# Step 7: Evaluate the Model
def evaluate_model_on_datasets(model, processor, datasets, num_samples=100):
    results = {}
    for name, dataset in datasets.items():
        total_cer = 0
        num_samples = min(num_samples, len(dataset))
        for i in range(num_samples):
            waveform, sample_rate, _, _, _, _ = dataset[i]
            transcription = speech_to_text(model, processor, waveform, sample_rate)
            reference = dataset[i][2]
            total_cer += cer(reference, transcription)
        average_cer = total_cer / num_samples
        results[name] = average_cer
        print(f"Average CER for {name}: {average_cer}")
    return results

results = evaluate_model_on_datasets(model, processor, datasets)

# Print the results of the evaluation
print(results)

Average CER for train-clean-100: 0.002039098849354967
Average CER for train-clean-360: 0.001401399099727044
Average CER for test-clean: 0.008527167264691528
Average CER for test-other: 0.011143117364111898
{'train-clean-100': 0.002039098849354967, 'train-clean-360': 0.001401399099727044, 'test-clean': 0.008527167264691528, 'test-other': 0.011143117364111898}
